In [1]:
import numpy as np

There is a question - can we use a dumb laser rangefinder (or something similar) to do fish length detection?

Let's try to answer this with a laser pointer.  This solution uses a single laser pointers to project a dot onto a surface that can then be imaged.  Due to perspective, the corresponding pixel location of the laser dot changes with the distance to the surface, so we can estimate the distance of the surface.

Let's set the stage for this.  For all simulations, we will use a pinhole camera model.  We'll make life easy and assume an Olympus TG-6 at the widest angle setting, and ignore the air-lens-water interface.

Thus, we have the following parameters:
- Sensor size (width x height): 4000 x 3000
- Pixel Pitch (um): 1.5
- Focal Length (mm): 4.5

Let's place one lasers pointed parallel to the optical axis with a baseline separation of 50 mm to the left of the optical axis.  We will define the global origin on center of the sensor, with $x$ along the width of the sensor, $y$ along the height of the sensor, and $z$ pointing out of the camera into the lens.

In [2]:
sensor_size_px = np.array([4000, 3000])
pixel_pitch_mm = 0.0015
focal_length_mm = 4.5
laser_1_origin = np.array([0.05, 0.05, 0])
laser_1_axis = np.array([0, 0, 1])

Let's assume we have a flat object 1 m from the sensor.  The lasers will project one dot onto the object.  Specifically, we will have a dot at [-0.050, 0, 1].

In [3]:
plane_normal = np.array([0, 0, 1])
plane_origin = np.array([0, 0, 5])

For a plane defined as $\left<\left(\mathbf{p} - \mathbf{p_0}\right), \mathbf{n}\right> = 0$ and a vector defined as $\mathbf{p} = \mathbf{l_0} + \mathbf{l}d, d \in \mathbb{R}$, the point of intersection is the point on the vector such that $d = \frac{\left<\left(\mathbf{p_0} - \mathbf{l_0}\right), \mathbf{n}\right>}{\left<\mathbf{l}, \mathbf{n}\right>}$

In [4]:
laser_1_scalar = np.dot((plane_origin - laser_1_origin), plane_normal) / np.dot(laser_1_axis, plane_normal)
laser_1_dot = laser_1_origin + laser_1_axis * laser_1_scalar
laser_1_dot

array([0.05, 0.05, 5.  ])

Any point $[x_1, x_2, x_3]$ in the world in the field of view will project onto the image plane as follows:
$$ \begin{pmatrix}y_1\\y_2\end{pmatrix} = -\frac{f}{x_3}\begin{pmatrix}x_1\\x_2\end{pmatrix} $$

In [5]:
laser_1_projection = -focal_length_mm / 1e3 / laser_1_dot[2] * laser_1_dot[0:2] / (pixel_pitch_mm / 1e3) # in pixels
laser_1_projection

array([-30., -30.])

In [6]:
def image_coordinate_to_projected_point(S):
    assert isinstance(S, np.ndarray)
    # S += sensor_size_px / 2
    I_project = S * pixel_pitch_mm / 1e3
    I = np.array([I_project[0], I_project[1], -focal_length_mm / 1e3])
    return I


In [7]:
projected_point = image_coordinate_to_projected_point(laser_1_projection)
projected_point

array([-4.5e-05, -4.5e-05, -4.5e-03])

In [8]:
final_laser_axis = -projected_point / np.linalg.norm(projected_point)
final_laser_axis

array([0.009999  , 0.009999  , 0.99990001])

Let $\ell$ be the laser origin, $\alpha$ be the laser axis, $v$ be the final laser axis, $p$ be the real world laser point

Because we know both are the laser line, they should intersect.

Thus, we can define the following constants:

$$k_1 | k_1 \alpha + \ell = p$$
$$k_2 | k_2 v = p$$

Thus, we can break this down into three equations with two unknowns.

$$k_1 \alpha + \ell = k_2 v$$

Thus:
$$ k_1 a_x + \ell_x = k_2 v_x$$
$$ k_1 a_z = k_2 v_z$$
$$ k_1 = \frac{k_2 v_z}{a_z}$$
$$a_x \left(\frac{k_2 v_z}{a_z}\right) + \ell_x = k_2 v_x$$
$$\left(\frac{a_x v_z}{a_z}\right) k_2 - k_2 v_x = -\ell_x$$
$$k_2 \left(\frac{a_x v_z}{a_z} - v_x\right) = -\ell_x$$
$$k_2 = \frac{-\ell_x}{\frac{a_x v_z}{a_z} - v_x} = \frac{-a_z \ell_x}{a_x v_z - a_z v_x}$$

In [9]:
k_2 = (-laser_1_axis[2] * laser_1_origin[0]) / (laser_1_axis[0] * final_laser_axis[2] - laser_1_axis[2] * final_laser_axis[0])
k_2

5.0004999750025005

In [10]:
world_point = k_2 * final_laser_axis
world_point

array([0.05, 0.05, 5.  ])

In [11]:
laser_1_origin

array([0.05, 0.05, 0.  ])

$$\begin{bmatrix}
\alpha_x & -v_x\\
\alpha_y & -v_y\\
\alpha_z & -v_z
\end{bmatrix} \begin{bmatrix}
k_1\\
k_2
\end{bmatrix} = \begin{bmatrix}
- \ell_x\\
- \ell_y\\
0
\end{bmatrix}$$

$$A^T A = \begin{bmatrix}
\alpha_x & \alpha_y & \alpha_z\\
-v_x & -v_y & -v_z
\end{bmatrix} \begin{bmatrix}
\alpha_x & -v_x\\
\alpha_y & -v_y\\
\alpha_z & -v_z
\end{bmatrix} = \begin{bmatrix}
\alpha_x^2 + \alpha_y^2 + \alpha_z^2 & -\alpha_x v_x - \alpha_y v_y - \alpha_z v_y\\
-\alpha_x v_x - \alpha_y v_y - \alpha_z v_y & v_x^2 + v_y^2 + v_z^2
\end{bmatrix} = \begin{bmatrix}
||\alpha||^2 & -\alpha^T v\\
-\alpha^T v & ||v||^2
\end{bmatrix}$$

Because $\alpha$ and $v$ are unit vectors

$$A^T A = \begin{bmatrix}
1 & -\alpha^T v\\
-\alpha^T v & 1
\end{bmatrix}$$

$$A^T \ell = \begin{bmatrix}
\alpha_x & \alpha_y & \alpha_z\\
-v_x & -v_y & -v_z
\end{bmatrix} \begin{bmatrix}
- \ell_x\\
- \ell_y\\
0
\end{bmatrix} = \begin{bmatrix}
- \ell_x \alpha_x + -\ell_y \alpha_y\\
\ell_x v_x + \ell_y v_y
\end{bmatrix} = \begin{bmatrix}
-\alpha^T \ell\\
v^T \ell
\end{bmatrix}$$

$$\begin{bmatrix}
1 & -\alpha^T v & -\alpha^T \ell\\
-\alpha^T v & 1 & v^T \ell
\end{bmatrix} \sim \begin{bmatrix}
1 & - \alpha^T v & -\alpha^T \ell\\
-1 & \frac{1}{\alpha^T v} & \frac{v^T \ell}{\alpha^T v}
\end{bmatrix} \sim \begin{bmatrix}
1 & - \alpha^T v & -\alpha^T \ell\\
0 & \frac{1}{\alpha^T v} - \alpha^T v & -\alpha^T \ell + \frac{v^T \ell}{\alpha^T v}
\end{bmatrix}$$

$$\sim \begin{bmatrix}
1 & -\alpha^T v & -\alpha^T \ell\\
0 & \frac{1}{\alpha^T v} - \alpha^T v & -\alpha^T \ell + \frac{v^T \ell}{\alpha^T v}
\end{bmatrix} \sim \begin{bmatrix}
1 & - \alpha^T v & -\alpha^T \ell\\
0 & 1 & \left( -\alpha^T \ell + \frac{v^T \ell}{\alpha^T v} \right) \left( \frac{1}{\frac{1}{\alpha^T v}} - \alpha^T v \right)
\end{bmatrix}$$

$$k_2 = \left( -\alpha \ell + \frac{v^T \ell}{\alpha^T v} \right) \left( \frac{1}{\frac{1}{\alpha^T v} - \alpha^T v} \right)$$
$$ = \frac{-\alpha^T \ell \alpha^T v + v^T \ell}{\alpha^T v} \frac{\alpha^T v}{1 - \alpha^T v \alpha^T v}$$
$$ = \frac{-\alpha^T \ell \alpha^T v + v^T \ell}{1 - (\alpha^T v)^2}$$

In [12]:

k2_ls = ((final_laser_axis.T @ laser_1_origin) - ((laser_1_axis.T @ laser_1_origin) * (laser_1_axis.T @ final_laser_axis))) / \
                    (1 - (laser_1_axis.T @ final_laser_axis) ** 2)

k2_ls

5.000499975003039

In [13]:
world_point = k2_ls * final_laser_axis
world_point

array([0.05, 0.05, 5.  ])

In [14]:
laser_1_origin

array([0.05, 0.05, 0.  ])